# WildNav with DINOv2 + LightGlue on GPU

This notebook runs the WildNav localization pipeline using DINOv2 feature extraction and LightGlue matching on Google Colab's GPU.

**Before running:** Make sure to enable GPU in Runtime > Change runtime type > Hardware accelerator > GPU (T4)

## 1. Check GPU Availability

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: GPU not available! Enable it in Runtime > Change runtime type")

## 2. Clone Repository from GitHub

In [ ]:
!git clone https://github.com/SiddharthSirohi/betterwildnav.git
%cd betterwildnav

## 3. Install Dependencies

In [ ]:
# Install PyTorch with CUDA support (if not already installed)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install project dependencies
!pip install -q numpy opencv-python-headless matplotlib pandas scikit-learn seaborn haversine requests

# Install LightGlue
!pip install -q git+https://github.com/cvg/LightGlue.git

# Install Kornia (required by LightGlue)
!pip install -q kornia kornia-rs

print("✅ All dependencies installed!")

## 4. Verify Installation

In [ ]:
import sys
sys.path.append('/content/betterwildnav/wildnav/src')

# Test imports
from lightglue import LightGlue, SuperPoint
from lightglue.utils import load_image
print("✅ LightGlue imported successfully")

# Test DINOv2 import
import torch
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
print("✅ DINOv2 loaded successfully")
del model  # Free memory

## 5. Upload Your Data (Optional)

If you want to test with your own drone images and satellite maps, upload them here.
Otherwise, the repository should already contain sample data.

In [ ]:
# Uncomment to upload files
# from google.colab import files
# uploaded = files.upload()

# Check if data exists
import os
data_path = '/content/betterwildnav/wildnav/assets'
if os.path.exists(data_path):
    print(f"✅ Data directory found: {data_path}")
    print(f"   Query images: {len(os.listdir(data_path + '/query'))} files")
    print(f"   Map images: {len(os.listdir(data_path + '/map'))} files")
else:
    print("⚠️ WARNING: Data directory not found!")
    print("You may need to upload your data.")

## 6. Run DINOv2 Feature Extraction Test

Quick test to verify DINOv2 works with GPU acceleration.

In [ ]:
%cd /content/betterwildnav/wildnav/src

import torch
import time
from lightglue.utils import load_image
from dinov2_extractor import DINOv2Extractor

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Initialize DINOv2 extractor
print("\nInitializing DINOv2...")
extractor = DINOv2Extractor(
    model_name='dinov2_vits14',
    max_keypoints=2048,
    device=device
)

# Load test image
test_image_path = '../assets/map/1_query_image.png'
if not os.path.exists(test_image_path):
    print(f"⚠️ Test image not found: {test_image_path}")
else:
    image_tensor = load_image(test_image_path).to(device)
    print(f"Image loaded: {image_tensor.shape}")

    # Extract features with timing
    print("\nExtracting features...")
    start_time = time.time()
    features = extractor.extract(image_tensor)
    elapsed = time.time() - start_time

    print(f"\n✅ Feature extraction complete in {elapsed:.2f}s")
    print(f"   Keypoints: {features['keypoints'].shape}")
    print(f"   Descriptors: {features['descriptors'].shape}")
    print(f"   Scores: {features['scores'].shape}")
    
    if device == 'cuda':
        print(f"\n🚀 GPU speedup is significant! (CPU would take ~{elapsed*10:.0f}s)")

## 7. Run Full Localization Pipeline

This runs the complete WildNav pipeline with DINOv2 + LightGlue.

In [ ]:
%cd /content/betterwildnav/wildnav/src

import time
start_time = time.time()

# Run the main pipeline
!python wildnav.py

elapsed = time.time() - start_time
print(f"\n\n{'='*60}")
print(f"Total processing time: {elapsed/60:.2f} minutes")
print(f"{'='*60}")

## 8. View Results

In [ ]:
import pandas as pd
from IPython.display import Image, display
import os

# Display results CSV
results_path = '/content/betterwildnav/wildnav/results/calculated_coordinates.csv'
if os.path.exists(results_path):
    print("📊 Localization Results:\n")
    df = pd.read_csv(results_path, header=None)
    df.columns = ['Image', 'True_Lat', 'True_Lon', 'Calc_Lat', 'Calc_Lon', 
                  'Error_Lat', 'Error_Lon', 'Error_m', 'Col8', 'Col9', 'Col10']
    print(df[['Image', 'Error_m']].to_string(index=False))
    print(f"\n📍 Mean Absolute Error: {df['Error_m'].mean():.2f} meters")
else:
    print("⚠️ Results file not found")

# Display visualizations
print("\n🖼️ Localization Visualizations:\n")
results_dir = '/content/betterwildnav/wildnav/results'
if os.path.exists(results_dir):
    for img_file in sorted(os.listdir(results_dir)):
        if img_file.endswith('_located.png'):
            print(f"\n{img_file}:")
            display(Image(filename=os.path.join(results_dir, img_file), width=800))

## 9. Download Results

In [ ]:
from google.colab import files
import shutil

# Create a zip of results
!cd /content/betterwildnav/wildnav && zip -r results.zip results/

# Download
files.download('/content/betterwildnav/wildnav/results.zip')
print("✅ Results downloaded!")

## Performance Notes

- **GPU (T4)**: ~2-5 seconds per image for DINOv2 extraction
- **CPU**: ~30-60 seconds per image for DINOv2 extraction
- **Speedup**: 10-20x faster on GPU

For production use, GPU is **strongly recommended** for DINOv2-based localization.